In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
%matplotlib inline 
import time
import cv2
import seaborn as sns
from matplotlib import pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import os

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model
from keras.layers import Embedding, LSTM, Dropout, Dense, Input, Bidirectional, Flatten, Conv2D, MaxPooling2D, concatenate, Conv1D, MaxPooling1D, GlobalMaxPool1D
import keras.backend as K
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping

In [ ]:
from tensorflow import keras
from keras.applications.resnet50 import ResNet50

from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from gensim.models import KeyedVectors
from tensorflow import keras
from keras.applications.resnet50 import ResNet50
from keras.applications import DenseNet201
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, add
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.merge import concatenate
from tensorflow.keras.utils import Sequence


In [ ]:
!pip install nagisa
import nagisa
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
%tensorflow_version 2.x
from keras import utils, layers, models, optimizers
from keras.preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
densenet_weights_path = '../input/densenet201-imagenet-pretrained-weights/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
X_train = pd.read_csv("../input/rakuten-multimodal-colour-extraction/X_train.csv", index_col=0)
y_train = pd.read_csv("../input/rakuten-multimodal-colour-extraction/y_train.csv", index_col=0)
X_test = pd.read_csv("../input/rakuten-multimodal-colour-extraction/X_test.csv", index_col=0)
train_clean = pd.read_csv('../input/rakuten-multimodal-colour-extraction/train_clean.csv')
test_clean = pd.read_csv('../input/rakuten-multimodal-colour-extraction/test_clean.csv')
separated_X_train = pd.read_csv('../input/d/xianlili/vectorized-x-train/separated_X_train.csv')
separated_X_test = pd.read_csv('../input/d/xianlili/vectorized-x-train/separated_X_test.csv')
vectorized_X_test = pd.read_csv('../input/d/xianlili/vectorized-x-train/vectorized_X_test.csv', index_col=0)
vectorized_X_train = pd.read_csv('../input/d/xianlili/vectorized-x-train/vectorized_X_train.csv', index_col=0)
predict = pd.read_csv('../input/predict/predict.csv')

In [ ]:
X_test.shape

In [ ]:
vectorized_X_train["vectorized item name"] = vectorized_X_train["vectorized item name"].apply(lambda x : x.strip("[]")).apply(lambda x : x.split(', ')).apply(lambda x : [float(element) for element in x] )
vectorized_X_train["vectorized item description"] = vectorized_X_train["vectorized item description"].apply(lambda x : x.strip("[]")).apply(lambda x : x.split(', ')).apply(lambda x : [float(element) for element in x] )

In [ ]:
vectorized_X_test["vectorized item name"] = vectorized_X_test["vectorized item name"].apply(lambda x : x.strip("[]")).apply(lambda x : x.split(', ')).apply(lambda x : [float(element) for element in x] )
vectorized_X_test["vectorized item description"] = vectorized_X_test["vectorized item description"].apply(lambda x : x.strip("[]")).apply(lambda x : x.split(', ')).apply(lambda x : [float(element) for element in x] )

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

DIM = 256
IMG_DIM = (DIM, DIM)
IMG_SHAPE = (DIM, DIM, 3)
IMG_PREFIX  = "../input/rakuten-multimodal-colour-extraction/images/images/"

In [ ]:
train = pd.concat([
    X_train,y_train,vectorized_X_train,train_clean,separated_X_train
], axis=1)

In [ ]:
train = train.loc[:,~train.columns.duplicated()]

In [ ]:
train.isna().sum()

In [ ]:
train = train.dropna()

In [ ]:
train = train.reset_index(drop=True)

In [ ]:
test = pd.concat([
    X_test,vectorized_X_test,test_clean,separated_X_test,y_train
], axis=1)

In [ ]:
test = test.loc[:,~test.columns.duplicated()]

In [ ]:
test = test.iloc[:37347,:]

In [ ]:
test.shape

In [ ]:
#test = test.dropna()

In [ ]:
test.shape

In [ ]:
df_rakuten = train.copy()

index_train = int(train.shape[0]*0.7)
index_val = int(train.shape[0]*0.85)

In [ ]:
df_train = df_rakuten.iloc[:index_train, :]
df_val = df_rakuten.iloc[index_train:index_val, :]


In [ ]:
test.shape

In [ ]:
df_train.shape

In [ ]:
df_val.shape

In [ ]:
def encode(encoder, colors):
    # Encode colors to numerical array
    indexes = encoder.transform(colors)
    
    # Create an array with 0s
    encoded = np.zeros(len(encoder.classes_))
    
    # Put 1s at the specified indexes
    encoded[indexes] = 1
    
    return encoded

def decode(encoder, encoded):
    # Decode the encoded
    decoded = np.where(encoded == 1)[0].ravel()
            
    # Get the colors
    return encoder.inverse_transform(decoded)

def extract_colors(col):
    colors = set()
    
    # Loop over the rows
    for row in col.values:
        # Some cleaning & list transformation
        row = row[1:-1].split(', ')
        # Loop over the elements of the list
        for color in row:
            # Quick clean
            color = color.replace("'", "")
            # Add to the set
            colors.add(color)
        
    return list(colors)

clean_tags = lambda x: [e.replace("'", "")  for e in x[1:-1].split(', ')]

In [ ]:
%%time

# Get colors
colors = extract_colors(df_train['color_tags'])

# Prepare the encoder for color tags
encoder = LabelEncoder()
encoder = encoder.fit(colors)

In [ ]:
%%time

# Clean
df_train["color_tags"] = df_train["color_tags"].apply(clean_tags)

# Create column encoded which are the encoded colors
df_train["encoded_colors"] = df_train["color_tags"].apply(lambda x: encode(encoder, x))

# Clean
df_val["color_tags"] = df_val["color_tags"].apply(clean_tags)

# Create column encoded which are the encoded colors
df_val["encoded_colors"] = df_val["color_tags"].apply(lambda x: encode(encoder, x))

# Clean
test["color_tags"] = test["color_tags"].apply(clean_tags)

# Create column encoded which are the encoded colors
test["encoded_colors"] = test["color_tags"].apply(lambda x: encode(encoder, x))

In [ ]:
df_train["encoded_colors"] = df_train["encoded_colors"].apply(lambda x : [int(element) for element in x] )
df_val["encoded_colors"] = df_val["encoded_colors"].apply(lambda x : [int(element) for element in x] )
test["encoded_colors"] = test["encoded_colors"].apply(lambda x : [int(element) for element in x] )

In [ ]:
print(len(df_train))
df_train = df_train[df_train["color_tags"].apply(len) < 4]
print(len(df_train))

In [ ]:
print(len(df_val))
df_val = df_val[df_val["color_tags"].apply(len) < 4]
print(len(df_val))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(df_train["color_tags"].tolist())
mlb.classes_

In [ ]:
train_labels = mlb.transform(df_train["color_tags"].tolist())
train_labels.shape

In [ ]:
val_labels = mlb.transform(df_val["color_tags"].tolist())
val_labels.shape

In [ ]:
test_labels = mlb.transform(test["color_tags"].tolist())
test_labels.shape

In [ ]:
test_labels

In [ ]:
df_train = df_train.drop(columns = ['item_name', 'item_caption', 'color_tags', 'item_description','Unnamed: 0','separated item name', 'separated item description'])

In [ ]:
df_val = df_val.drop(columns = ['item_name', 'item_caption', 'color_tags', 'item_description','Unnamed: 0','separated item name', 'separated item description'])


In [ ]:
test = test.drop(columns = ['item_name', 'item_caption', 'color_tags', 'item_description','Unnamed: 0','separated item name', 'separated item description'])

In [ ]:
test.head()

In [ ]:
#predict = predict.drop(columns = ['Unnamed: 0'])

In [ ]:
df_predict = predict.copy()

In [ ]:
#df_predict["colors_tags"] = df_predict["colors"].apply(lambda x: decode(encoder, x))

In [ ]:
#df_predict["color_tags"]

In [ ]:
from keras import backend as K
#IMPLEMENTATION DE LA METRIC UTILISEE PAR LE CONCOURS F1

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
class CustomSequenceGenerator(Sequence):

    def __init__(self, df, dim=DIM, batch_size=10,
                 n_classes=19, n_channels=3, vec_size=64, shuffle=True):
        # Keras generator
        self.image_dir = IMG_PREFIX
        self.df = df
        self.batch_size = batch_size
        #self.csv_file = pd.read_csv(csv_file_path)
        self.n_classes = n_classes
        self.dim = dim
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.vec_size = vec_size

        #self.csv_file.set_index('File', inplace=True, drop=True)

    def __len__(self):
        """It is mandatory to implement it on Keras Sequence"""
        return int(np.ceil(self.df.shape[0] / float(self.batch_size)))

    def __getitem__(self, index):

        # Generate indexes of the batch
        samples = self.df.values[index * self.batch_size : (index + 1) * self.batch_size, : ]

        x, y = self.__data_generation(samples, index)
        return x, y

    def __data_generation(self, samples, start_index):

        x_batch_image = np.empty((self.batch_size, self.dim, self.dim, self.n_channels))
        x_batch_vector_name = np.empty((self.batch_size, self.vec_size))
        x_batch_vector_description = np.empty((self.batch_size, self.vec_size))
        y_batch = np.empty((self.batch_size, self.n_classes))
        #self.csv_file.reindex()
        for i, sample in enumerate(samples):
            image_file_path = self.image_dir + sample[0]
            image = keras.preprocessing.image.load_img(image_file_path, target_size=IMG_DIM)

            #features, labels = self.preprocess_csv(self.csv_file, sample, self.labels_dict, self.n_classes)
            
            x_batch_image[i] = image
            x_batch_vector_name[i] = sample[1]
            x_batch_vector_description[i] = sample[2]
            y_batch[i] = sample[3]

        return [x_batch_image, x_batch_vector_name, x_batch_vector_description], y_batch
    
train_generator = CustomSequenceGenerator(df_train)
val_generator = CustomSequenceGenerator(df_val)
test_generator =  CustomSequenceGenerator(test)

In [ ]:
from keras.applications import DenseNet201
from keras import optimizers
from tensorflow import keras

def compile_model():
    
    vectorisation_inputs_name = Input(shape = (64,))
    x1 = Dense(64, activation='relu')(vectorisation_inputs_name)
    x1 = Dropout(0.3)(x1)
    x1 = Dense(64, activation = "relu")(x1)
    x1 = Dropout(0.3)(x1)
    vectorisation_out_name = Dense(18, activation = 'relu')(x1)
    
    vectorisation_inputs_description = Input(shape = (64,))
    x2 = Dense(64, activation='relu')(vectorisation_inputs_description)
    x2 = Dropout(0.3)(x2)
    x2 = Dense(64, activation = "relu")(x2)
    x2 = Dropout(0.3)(x2)
    vectorisation_out_description = Dense(18, activation = 'relu')(x2)

    cnn_input = Input(shape=IMG_SHAPE)
    y = DenseNet201(include_top = False, pooling = 'avg', weights=densenet_weights_path)(cnn_input)
    cnn_out = Dense(512, activation='relu')(y)

    concat_inp = concatenate([cnn_out, vectorisation_out_name, vectorisation_out_description])
    z = Dense(256, activation='relu')(concat_inp)
    z = Dropout(0.3)(z)
    z = Dense(128, activation='relu')(z)
    z = Dropout(0.3)(z)
    output = Dense(train_labels.shape[1], activation='sigmoid')(z)

    model = Model(inputs=[cnn_input, vectorisation_inputs_name, vectorisation_inputs_description], outputs=[output])
    adam = Adam(lr=0.001, decay=1e-5)

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy',f1])
    
    return model
  
model = compile_model()
model.summary()

In [ ]:
model.layers[9].trainable = False #freeze DenseNet201

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("../input/model-densetext/save_at_{epoch}.h5")]
es = EarlyStopping(monitor='val_loss',patience=4)

fit_history = model.fit_generator(
        train_generator,
        epochs = 15,
        validation_data=val_generator,
        callbacks = [callbacks,es]
)

model.save('model_image_text.h5')

In [ ]:
import tensorflow as tf

In [ ]:
dependencies = {
    'f1': f1
}


In [ ]:
model = tf.keras.models.load_model("../input/model-densetext/model_image_text.h5", custom_objects=dependencies)

In [ ]:
model.summary()

In [ ]:
%%time
predict=model.predict(test_generator)

In [ ]:
predict

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
df_predict = pd.DataFrame(data=predict)

In [ ]:
df_predict

In [ ]:
df_predict.to_csv('./predict.csv')

In [ ]:
df_predict = df_predict.drop(columns = ['Unnamed: 0'])

In [ ]:
df_predict[df_predict > 0.2] = 1

In [ ]:

df_predict[df_predict < 0.2] = 0

In [ ]:
df_predict.columns = ['Beige', 'Black', 'Blue', 'Brown', 'Burgundy', 'Gold', 'Green',
       'Grey', 'Khaki', 'Multiple Colors', 'Navy', 'Orange', 'Pink',
       'Purple', 'Red', 'Silver', 'Transparent', 'White', 'Yellow']

In [ ]:
df_predict['color_tags'] = df_predict.values.tolist()

In [ ]:
df_predict["colors"] = df_predict["colors"].apply(lambda x : [int(element) for element in x] )

In [ ]:
loss     = model.history['loss']
val_loss = model.history['val_loss']
acc      = model.history['accuracy']
val_acc  = model.history['val_accuracy']
f1       = model.history['f1']
val_f1   = model.history['val_f1']

# Visualize the history plots
plt.figure()
plt.plot(loss, 'b', label='Training loss')
plt.plot(val_loss, 'm', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
plt.figure()
plt.plot(acc, 'b', label='Training acc')
plt.plot(f1, 'v', label='Training f1')
plt.plot(val_acc, 'm', label='Validation acc')
plt.plot(val_f1, 'y', label='Validation f1')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()